<a href="https://colab.research.google.com/github/zahranorozzadeh/tamrin49/blob/main/jalase49.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import tensorflow as tf
from tensorflow.keras.layers  import Dense,Flatten,Conv2D

In [55]:
print(tf.__version__)

2.6.0


#Preparev Data

In [56]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [57]:
y_train[:20]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9],
      dtype=uint8)

In [58]:
x_train.shape

(60000, 28, 28)

In [59]:
#add a channels dimension

x_train = x_train[...,tf.newaxis].astype("float32")
x_test = x_test[...,tf.newaxis].astype("float32")

In [60]:
x_train.shape

(60000, 28, 28, 1)

In [61]:
train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

#Define Model


In [62]:
class MyModel(tf.keras.Model):
  def __init__(self,number_of_classes):
    super().__init__()
    self.Conv1 = Conv2D(32,(3,3), activation='relu')
    self.Flatten = Flatten()
    self.fc1 = Dense(128, activation = 'relu')
    self.fc2 = Dense(number_of_classes)
  
  def call(self , x):
    #x.shape = 28 * 28

    y = self.Conv1(x)
    #y.shape = 32 * 26 * 26

    w = self.Flatten(y)
    #w.shape = 1 * 21632

    z = self.fc1(w)
    #z.shape = 1 * 128

    output = self.fc2(z)
    #output.shape = 1 * 10

    return output

model = MyModel(10)

In [63]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [64]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_acc')


test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_acc')

In [75]:
def train_step(images,y) :
  with tf.GradientTape() as tape:
      y_pred = model(images,training=True)
      loss = loss_function(y,y_pred)

  #calculate gradients
  gradients = tape.gradient(loss,model.trainable_variables)


  #update
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))



  train_loss(loss)
  train_acc(y,y_pred)


In [76]:
def test_step(images, y):
  y_pred = model(images, training = False)
  loss = loss_function(y,y_pred)

  test_loss(loss)
  test_acc(y,y_pred)

In [77]:
from tqdm import tqdm 

In [78]:
def train():
  epochs = 5
  for epoch in  range(epochs):
      train_loss.reset_states()
      train_acc.reset_states()
      test_loss.reset_states()
      test_acc.reset_states()

      for images,labels in tqdm(train_data):
         train_step(images,labels)
      for images,labels in tqdm(test_data):
          test_step(images,labels)

      print('epoch:', epoch + 1)
      print('loss:', train_loss.result())
      print('accuracy:',train_acc.result())
      print('val loss:',test_loss.result())
      print('val accuracy:',test_acc.result())

In [79]:
#fit
train()

100%|██████████| 313/313 [00:02<00:00, 114.61it/s]


epoch: 1
loss: tf.Tensor(0.13492651, shape=(), dtype=float32)
accuracy: tf.Tensor(0.95951664, shape=(), dtype=float32)
val loss: tf.Tensor(0.064073384, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.9794, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 117.39it/s]


epoch: 2
loss: tf.Tensor(0.041393403, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9873833, shape=(), dtype=float32)
val loss: tf.Tensor(0.052217543, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.9822, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 113.75it/s]


epoch: 3
loss: tf.Tensor(0.021909874, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9932, shape=(), dtype=float32)
val loss: tf.Tensor(0.053558093, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.9836, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 114.98it/s]


epoch: 4
loss: tf.Tensor(0.014073599, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9953667, shape=(), dtype=float32)
val loss: tf.Tensor(0.056068268, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.9833, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 113.42it/s]

epoch: 5
loss: tf.Tensor(0.008844411, shape=(), dtype=float32)
accuracy: tf.Tensor(0.99693334, shape=(), dtype=float32)
val loss: tf.Tensor(0.06883855, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.9831, shape=(), dtype=float32)


In [30]:
# class Person:
#   def __init__(self,n):
#     self.name = n
#   def __call__(self,x):
#     print('test')

#   def my_function(self):
#     print('test')

# p1 = Person('zahra')
# p2 = Person('reza')


In [29]:
# p1(13)

In [28]:
# p1.my_function()